In [1]:
"""
Title: Knowledge Distillation
Author: [Kenneth Borup](https://twitter.com/Kennethborup)
Date created: 2020/09/01
Last modified: 2020/09/01
Description: Implementation of classical Knowledge Distillation.
"""

"""
## Introduction to Knowledge Distillation
Knowledge Distillation is a procedure for model
compression, in which a small (student) model is trained to match a large pre-trained
(teacher) model. Knowledge is transferred from the teacher model to the student
by minimizing a loss function, aimed at matching softened teacher logits as well as
ground-truth labels.
The logits are softened by applying a "temperature" scaling function in the softmax,
effectively smoothing out the probability distribution and revealing
inter-class relationships learned by the teacher.
**Reference:**
- [Hinton et al. (2015)](https://arxiv.org/abs/1503.02531)
"""

"""
## Setup
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
"""
## Prepare the dataset
The dataset used for training the teacher and distilling the teacher is
[MNIST](https://keras.io/api/datasets/mnist/), and the procedure would be equivalent for any other
dataset, e.g. [CIFAR-10](https://keras.io/api/datasets/cifar10/), with a suitable choice
of models. Both the student and teacher are trained on the training set and evaluated on
the test set.
"""

# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
from skimage.transform import resize

def resize_img(data):
    output = np.zeros((data.shape[0],64))
    for i, img in enumerate(data):
        output[i]=resize(img, (8,8)).flatten()
    return output

x_train_small = resize_img(x_train)
y_train_small = resize_img(y_train)
x_test_small = resize_img(x_test)
y_test_small = resize_img(y_test)


#bottle_resized = resize(bottle, (140, 54))

In [4]:
# Normalize data
x_train = x_train_small.astype("float32") * 16.0
x_train = np.reshape(x_train, (-1, 64))

x_test = x_test_small.astype("float32") * 16.0
x_test = np.reshape(x_test, (-1, 64))

In [5]:
np.max(x_test)

15.87451

Taken from [this tutorial](https://github.com/keras-team/keras-io/blob/master/examples/vision/knowledge_distillation.py)

In [6]:
"""
## Construct `Distiller()` class
The custom `Distiller()` class, overrides the `Model` methods `train_step`, `test_step`,
and `compile()`. In order to use the distiller, we need:
- A trained teacher model
- A student model to train
- A student loss function on the difference between student predictions and ground-truth
- A distillation loss function, along with a `temperature`, on the difference between the
soft student predictions and the soft teacher labels
- An `alpha` factor to weight the student and distillation loss
- An optimizer for the student and (optional) metrics to evaluate performance
In the `train_step` method, we perform a forward pass of both the teacher and student,
calculate the loss with weighting of the `student_loss` and `distillation_loss` by `alpha` and
`1 - alpha`, respectively, and perform the backward pass. Note: only the student weights are updated,
and therefore we only calculate the gradients for the student weights.
In the `test_step` method, we evaluate the student model on the provided dataset.
"""


class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.
        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                teacher_predictions,
                student_predictions,
            )
            loss = distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [7]:
"""
## Create student and teacher models
Initialy, we create a teacher model and a smaller student model. Both models are
convolutional neural networks and created using `Sequential()`,
but could be any Keras model.
"""

# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(64,)),
        layers.Dense(100, activation='relu'),
        layers.Dense(80, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(40, activation='relu'),
        layers.Dense(20, activation='relu'),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(64,)),
        layers.Dense(40, activation='relu'),
        layers.Dense(40, activation='relu'),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [8]:
"""
## Train the teacher
In knowledge distillation we assume that the teacher is trained and fixed. Thus, we start
by training the teacher model on the training set in the usual way.
"""

# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3218 - sparse_categorical_accuracy: 0.9029
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1587 - sparse_categorical_accuracy: 0.9521
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1299 - sparse_categorical_accuracy: 0.9608
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1142 - sparse_categorical_accuracy: 0.9649
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1004 - sparse_categorical_accuracy: 0.9694
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0934 - sparse_categorical_accuracy: 0.9710
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0838 - sparse_categorical_accuracy: 0.9744
Epoch 8/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0769 - sparse_categorical_accuracy: 0.9758
Epoch 9/10
1875/1875 [==========

In [9]:
teacher.optimizer.lr.numpy()

0.001

In [10]:
teacher.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1015 - sparse_categorical_accuracy: 0.9695


[0.10153129696846008, 0.9695000052452087]

In [11]:
"""
## Distill teacher to student
We have already trained the teacher model, and we only need to initialize a
`Distiller(student, teacher)` instance, `compile()` it with the desired losses,
hyperparameters and optimizer, and distill the teacher to the student.
"""

# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.MeanSquaredError(),
    alpha=0.1,
    temperature=10,
)
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
callback = keras.callbacks.LearningRateScheduler(scheduler)
# Distill teacher to student
distiller.fit(x_train, y_train, epochs=20, callbacks=[callback])

Epoch 1/20
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.8481 - student_loss: 0.5095 - distillation_loss: 6.1914 - lr: 0.0010
Epoch 2/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9215 - student_loss: 0.2712 - distillation_loss: 2.7063 - lr: 0.0010
Epoch 3/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9324 - student_loss: 0.2313 - distillation_loss: 2.1568 - lr: 0.0010
Epoch 4/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9384 - student_loss: 0.2093 - distillation_loss: 1.8848 - lr: 0.0010
Epoch 5/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9420 - student_loss: 0.1970 - distillation_loss: 1.7214 - lr: 0.0010
Epoch 6/20
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9452 - student_loss: 0.1864 - distillation_loss: 

In [12]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
callback = keras.callbacks.LearningRateScheduler(scheduler)
# Distill teacher to student
distiller.fit(x_train, y_train, epochs=20, callbacks=[callback])

Epoch 1/20
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9568 - student_loss: 0.1440 - distillation_loss: 1.1445 - lr: 3.6788e-04
Epoch 2/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9571 - student_loss: 0.1427 - distillation_loss: 1.1378 - lr: 3.6788e-04
Epoch 3/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9568 - student_loss: 0.1431 - distillation_loss: 1.1336 - lr: 3.6788e-04
Epoch 4/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9568 - student_loss: 0.1430 - distillation_loss: 1.1305 - lr: 3.6788e-04
Epoch 5/20
1875/1875 [==============================] - 3s 1ms/step - sparse_categorical_accuracy: 0.9572 - student_loss: 0.1418 - distillation_loss: 1.1236 - lr: 3.6788e-04
Epoch 6/20
1875/1875 [==============================] - 3s 2ms/step - sparse_categorical_accuracy: 0.9571 - student_loss: 0.1417 -

In [13]:
# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - sparse_categorical_accuracy: 0.9583 - student_loss: 0.1508


0.958299994468689

In [14]:
teacher.save("mnist8x8_100_80_60_40_20_10_eps1.h5")

In [15]:
student.save("mnist8x8_100_80_60_40_20_10_eps1-mirror.h5")

In [16]:
from tensorflow.python.keras.models import load_model
import onnx
import keras2onnx

In [17]:
model = load_model('mnist8x8_100_80_60_40_20_10_eps1.h5')
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, "mnist8x8_100_80_60_40_20_10_eps1.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 22 -> 17
The maximum opset needed by this model is only 9.


In [18]:
model = load_model('mnist8x8_100_80_60_40_20_10_eps1-mirror.h5')
onnx_model = keras2onnx.convert_keras(model, model.name)
onnx.save_model(onnx_model, "mnist8x8_100_80_60_40_20_10_eps1-mirror.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 13 -> 8
The maximum opset needed by this model is only 9.


Note: The student network was trained for a total of 30 epochs!


In [19]:
#point = [0.0, 0.01, 8.005, 14.005, 14.995, 13.995, 7.005, 0.01, 0.0, 0.0, 6.995, 7.995, 8.995, 14.995, 8.005, 0.0, 0.01, 0.0, 0.0, 0.01, 5.005, 13.995, 2.995, 0.0, 0.0, 0.0, 5.005, 9.005, 13.995, 13.005, 4.005, 0.0, 0.0, 1.005, 10.005, 14.005, 14.995, 12.005, 4.005, 0.01, 0.0, 0.0, 2.005, 13.005, 8.005, 1.995, 0.0, 0.01, 0.0, 0.0, 5.995, 14.995, 2.005, 0.01, 0.0, 0.01, 0.01, 0.0, 9.995, 9.995, 0.01, 0.01, 0.01, 0.01]
#point = [0.0, 0.01, 8.005, 14.005, 14.995, 13.995, 7.005, 0.01, 0.0, 0.0, 6.995, 7.995, 8.995, 14.995, 8.005, 0.0, 0.01, 0.0, 0.0, 0.01, 5.005, 13.995, 2.995, 0.0, 0.0, 0.0, 5.005, 9.005, 13.995, 13.005, 4.005, 0.0, 0.0, 1.005, 10.005, 14.005, 14.995, 12.005, 4.005, 0.01, 0.0, 0.0, 2.005, 13.005, 8.005, 1.995, 0.0, 0.01, 0.0, 0.0, 5.995, 14.995, 2.005, 0.01, 0.0, 0.01, 0.01, 0.0, 9.995, 9.995, 0.01, 0.01, 0.01, 0.01]
#point = [0.0, 0.01, 8.005, 14.005, 14.995, 13.995, 7.005, 0.01, 0.0, 0.0, 6.995, 7.995, 8.995, 14.995, 8.005, 0.0, 0.01, 0.0, 0.0, 0.01, 5.005, 13.995, 2.995, 0.0, 0.0, 0.0, 5.005, 9.005, 13.995, 13.005, 4.005, 0.0, 0.0, 1.005, 10.005, 14.005, 14.995, 12.005, 4.005, 0.01, 0.0, 0.0, 2.005, 13.005, 8.005, 1.995, 0.0, 0.01, 0.0, 0.0, 5.995, 14.995, 2.005, 0.01, 0.0, 0.01, 0.01, 0.0, 9.995, 9.995, 0.01, 0.01, 0.01, 0.01]
point = [0.1, 0.1, 7.95, 14.05, 14.95, 14.05, 6.95, 0.1, 0.0, 0.1, 6.95, 8.05, 9.05, 14.95, 8.05, 0.1, 0.1, 0.0, 0.1, 0.1, 5.05, 14.05, 2.95, 0.0, 0.1, 0.0, 4.95, 9.05, 14.05, 12.95, 4.05, 0.0, 0.0, 1.05, 9.95, 14.05, 14.95, 12.05, 4.05, 0.1, 0.1, 0.1, 2.05, 13.05, 8.05, 2.05, 0.0, 0.1, 0.0, 0.0, 6.05, 15.05, 2.05, 0.0, 0.1, 0.1, 0.1, 0.1, 9.95, 10.05, 0.1, 0.1, 0.1, 0.0]
teacher1 = load_model('mnist8x8_100_80_60_40_20_10.h5')
student1 = load_model('mnist8x8_100_80_60_40_20_10-mirror.h5')
p1=teacher1.predict(np.array([point]))
p2=student1.predict(np.array([point]))
print(np.argmax(p1)==np.argmax(p2))
print(p1)
print(p2)


OSError: SavedModel file does not exist at: mnist8x8_100_80_60_40_20_10.h5/{saved_model.pbtxt|saved_model.pb}

# Equivalence Properties
It seems that the `9200.e` properties with `e<5` are top 1 equivalent.
`9000*` and `9100*` on the other hand are not top 1 equivalent